# Data Exploration

Let's examine our ECG dataset

## Imports

In [ ]:
import glob

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

## Load Reference Data

In [ ]:
ref = pd.read_csv('../../datasets/cinc/REFERENCE.csv')

The reference data is a flat table that tells us what rhythm each record is in. `N` for normal sinus rhythm and `O` for other rhythm. Our task will be to classify ECG signals into these two classes

In [ ]:
ref.head()

Class ratio

We have roughly 1.5x more sinus rhythm data than other.

In [ ]:
ref.rhythm.value_counts()

The sampling rate for this dataset is 300 Hz.

In [ ]:
fs = 300

## Load Raw ECG Data + QRS locations

The data is stored in `npz` files. We are given both the raw ECG data as well as the QRS complex locations.

In [ ]:
data_dir = '../../datasets/cinc/'
files = sorted(glob.glob(data_dir + '*.npz'))
ecgs, qrs_inds = [], []
for f in files:
    with np.load(f) as npz:
        ecgs.append(npz['ecg'])
        qrs_inds.append(npz['qrs'])

Let's take a look at how long each ECG record is.

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(np.array(list(map(len, ecgs))) / fs);
plt.xlabel('Time (sec)')
plt.title('Length of ECG records')

Now let's look at the raw ECG data plotted with the QRS locations

### Normal sinus rhythm

By looking at sinus rhythm data we can get a sense of the noise level in the sensor without dealing with rhythm-induced artifacts.

In [ ]:
plt.figure(figsize=(10, 6))
ecg, qrs = ecgs[0], qrs_inds[0]
ts = np.arange(len(ecg)) / fs
plt.plot(ts, ecg)
plt.plot(ts[qrs], ecg[qrs], 'r.', ms=10, label='QRS Detections')
plt.xlim((12.5, 18.95))
plt.xlabel('Time (sec)')
plt.ylim((-.37, 1))
plt.title('Sinus Rhythm')

### Noisy sinus rhythm

Sometimes our ECG rhythm will be interrupted by transient noise artifacts, most likely caused by user error or motion artifact. Even in the presence of motion, the QRS detection algorithm still finds QRS complexes. In the presence of noise, there surely might be some error in the QRS detections.

In [ ]:
plt.figure(figsize=(10, 6))
ecg, qrs = ecgs[1], qrs_inds[1]
ts = np.arange(len(ecg)) / fs
plt.plot(ts, ecg)
plt.plot(ts[qrs], ecg[qrs], 'r.', ms=10, label='QRS Detections')
plt.xlim((16.74, 20.91))
plt.xlabel('Time (sec)')
plt.ylim((-0.68, 1.74))
plt.title('Noisy Sinus Rhythm')

### Atrial Fibrillation

This is an example of an atrial fibrillation ECG. Notice the irregularity in the QRS complex times. This will be the source of most of our features in the algorithm.

In [ ]:
plt.figure(figsize=(10, 6))
ecg, qrs = ecgs[3], qrs_inds[3]
ts = np.arange(len(ecg)) / fs
plt.plot(ts, ecg)
plt.plot(ts[qrs], ecg[qrs], 'r.', ms=10, label='QRS Detections')
plt.xlim((9, 16))
plt.xlabel('Time (sec)')
plt.ylim((-0.68, 1.6))
plt.title('AF Rhythm')

Download this notebook and play with the data yourself. Try to find snippets where the QRS detector messes up, or look for `other` records that have a regular rhythm. These are a few challenges that might confuse our algorithm.